In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import time
import os

from bs4 import BeautifulSoup
import re
import pandas as pd

import requests
import os
import urllib
from urllib.parse import urlparse

import subprocess


In [ ]:
IMPLICIT_DELAY = 60
EXPLICIT_DELAY = 30

In [ ]:
#setup option for chrome profile
chrome_options = Options()

## NEVER USE THE FOLLOWING OPTION SINCE IT WILL MAKE SUPPOSEDLY DIFFERENT WINDOWS FOR DIFFERENT CLASS MIXED IN SOME WAY!!!!
# chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
## the following option is to solve the error if using the above cause a problem
# chrome_options.add_argument("--remote-debugging-port=9222") 

prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument('--headless')
# chrome_options.add_argument('--start-maximized') # when used with headles enabled it is 800x600
# chrome_options.add_argument("--window-size=5120,2880") 

#start web driver
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(IMPLICIT_DELAY)
wait = WebDriverWait(driver, EXPLICIT_DELAY)

In [ ]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(driver, xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, wait, xpath):

    try:
        wait.until(ec.presence_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element("xpath",xpath)

def sync_get_elements_by_xpath(driver, wait, xpath):

    try:
        wait.until(ec.presence_of_all_elements_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_elements("xpath",xpath)

In [ ]:
root_address = 'https://learn.udacity.com/nanodegrees/nd880/parts/cd0569'
root_folder = './ai_for_trading_course_downloaded'

In [ ]:
# start
driver.get(root_address)
os.makedirs(root_folder, exist_ok=True)

In [ ]:
sync_get_element_by_xpath(driver, wait, "//div[text()='Sign in']").click()

email = sync_get_element_by_xpath(driver, wait, "//input[@id='email']")
password = sync_get_element_by_xpath(driver, wait, "//input[@id='revealable-password']")

email.send_keys("huohsien@gmail.com")
password.send_keys("Huo18941256")

sync_get_element_by_xpath(driver, wait, "//div[@data-testid='signin-form']//span[text()='Sign in']/..").click()


In [ ]:
program_name = sync_get_element_by_xpath(driver, wait, "//h1[@class='chakra-heading css-10cnqhb']").get_attribute("innerHTML")
course_block_elms = sync_get_elements_by_xpath(driver, wait, "//div[@class='css-pk1kta']")

In [ ]:
course_names = []
for course_block_elm in course_block_elms:
    course_name = sync_get_element_by_xpath(course_block_elm, wait,"//h2[@class='chakra-heading css-ltgw47']").get_attribute("innerHTML")
    course_names.append(course_name)
course_names


In [ ]:
e_header = sync_get_elements_by_xpath(driver, wait, "//div[@class='css-4nu87b']")
e_content = sync_get_elements_by_xpath(driver, wait, "//div[@class='css-dmdipe']")

header_height = int(e_header[0].get_attribute("scrollHeight"))
content_height = int(e_content[0].get_attribute("scrollHeight"))
full_page_height = content_height+header_height

In [ ]:
full_page_width = int(e_header[0].get_attribute("scrollWidth"))

In [ ]:
# from PIL import Image

# def fullpage_screenshot(driver, file):

#         print("Starting chrome full page screenshot workaround ...")

#         total_width = driver.execute_script("return document.body.offsetWidth")
#         total_height = driver.execute_script("return document.body.parentNode.scrollHeight")
#         viewport_width = driver.execute_script("return document.body.clientWidth")
#         viewport_height = driver.execute_script("return window.innerHeight")
#         print("Total: ({0}, {1}), Viewport: ({2},{3})".format(total_width, total_height,viewport_width,viewport_height))
#         rectangles = []

#         i = 0
#         while i < total_height:
#             ii = 0
#             top_height = i + viewport_height

#             if top_height > total_height:
#                 top_height = total_height

#             while ii < total_width:
#                 top_width = ii + viewport_width

#                 if top_width > total_width:
#                     top_width = total_width

#                 print("Appending rectangle ({0},{1},{2},{3})".format(ii, i, top_width, top_height))
#                 rectangles.append((ii, i, top_width,top_height))

#                 ii = ii + viewport_width

#             i = i + viewport_height

#         stitched_image = Image.new('RGB', (total_width, total_height))
#         previous = None
#         part = 0

#         for rectangle in rectangles:
#             if not previous is None:
#                 driver.execute_script("window.scrollTo({0}, {1})".format(rectangle[0], rectangle[1]))
#                 time.sleep(0.2)
#                 driver.execute_script("document.getElementById('topnav').setAttribute('style', 'position: absolute; top: 0px;');")
#                 time.sleep(0.2)
#                 print("Scrolled To ({0},{1})".format(rectangle[0], rectangle[1]))
#                 time.sleep(0.2)

#             file_name = "part_{0}.png".format(part)
#             print("Capturing {0} ...".format(file_name))

#             # driver.get_screenshot_as_file(file_name) // captured shots are inappropriate zoomed thus cropped
            
#             el = driver.find_element(By.TAG_NAME, "body")
#             el.screenshot(file_name)
            
            
#             screenshot = Image.open(file_name)

#             if rectangle[1] + viewport_height > total_height:
#                 offset = (rectangle[0], total_height - viewport_height)
#             else:
#                 offset = (rectangle[0], rectangle[1])

#             print("Adding to stitched image with offset ({0}, {1})".format(offset[0],offset[1]))
#             stitched_image.paste(screenshot, offset)

#             del screenshot
#             os.remove(file_name)
#             part = part + 1
#             previous = rectangle

#         stitched_image.save(file)
#         print("Finishing chrome full page screenshot workaround...")
#         return True

In [ ]:
# #setup option for chrome profile
# chrome_options = Options()

# ## NEVER USE THE FOLLOWING OPTION SINCE IT WILL MAKE SUPPOSEDLY DIFFERENT WINDOWS FOR DIFFERENT CLASS MIXED IN SOME WAY!!!!
# # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
# ## the following option is to solve the error if using the above cause a problem
# # chrome_options.add_argument("--remote-debugging-port=9222") 

# prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
# chrome_options.add_experimental_option("prefs", prefs)
# # chrome_options.add_argument('--headless')
# # chrome_options.add_argument('--start-maximized') # when used with headles enabled it is 800x600
# # chrome_options.add_argument("--window-size=5120,2880") 

# #start web driver
# driver = webdriver.Chrome(options=chrome_options)
# driver.implicitly_wait(IMPLICIT_DELAY)
# wait = WebDriverWait(driver, EXPLICIT_DELAY)
# driver.get("https://www.69shu.pro/txt/29612/20561845#google_vignette")

In [ ]:
driver.set_window_size(full_page_width,full_page_height)

el = driver.find_element(By.TAG_NAME, "body")
el.screenshot("main.png")


In [ ]:
# el = driver.find_element(By.TAG_NAME, "body")
# el.screenshot("main.png")

In [ ]:
# fullpage_screenshot(driver, "main.png")

In [ ]:
# driver.execute_script("return document.body.scrollHeight")


In [ ]:
# driver.execute_script("return document.body.offsetHeight")

In [ ]:
# element = driver.find_element(By.TAG_NAME, "body")

# element.get_attribute("scrollHeight")

In [ ]:
# # Prepare loop through all topics
# topic_elms = sync_get_elements_by_xpath("//*[@id='tree-core-curriculum']/*")
# num_topics = len(topic_elms)

# # iterate each topic
# for idx_topics in range(num_topics):
    
#     # Create topic folder
#     topic_name = ''.join(topic_elms[idx_topics].text.split('\n')[0:2])                   
#     os.makedirs(os.path.join(root_folder, topic_name), exist_ok=True)
    
#     debug_breakpoint("Topic {}".format(topic_name), prompt_message)
    
#     # click into a topic
#     link_topic = topic_elms[idx_topics].find_element_by_xpath("./a")
#     link_topic.click()
    
#     # Prepare loop through all lessons
#     expand_all_complete_lessons()
    
#     lesson_elms = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")
#     num_lessons = len(lesson_elms)
    
#     #iterate each lesson
#     for idx_lesson in range(num_lessons):

#         # Create lesson folder
#         lesson_elm = lesson_elms[idx_lesson]
#         lesson_name = ' - '.join(lesson_elm.text.split('\n')[1:3])                   
#         os.makedirs(os.path.join(root_folder, topic_name,lesson_name), exist_ok=True)

#         debug_breakpoint("Lesson {}".format(lesson_name), prompt_message)

#         # click into a lesson
#         link_lesson = lesson_elm.find_element_by_xpath("./div/div/a")
#         link_lesson.click()
        
#         # Prepare loop through all parts of a lesson
#         part_elms = sync_get_elements_by_xpath("//*[@id='tree-concepts']/ol/*")
#         num_parts = len(part_elms)
        
#         # iterate each part
#         for idx_part in range(num_parts):
            
#             # Create part folder
#             part_elm = part_elms[idx_part]
#             part_name = part_elm.text
#             os.makedirs(os.path.join(root_folder, topic_name,lesson_name, part_name), exist_ok=True)
            
#             debug_breakpoint("Lesson part {}".format(part_name), prompt_message)
            
#             # click into a part
#             link_part = part_elm.find_element_by_xpath("./a")
#             link_part.click()
            
#             # REAL CONTENTS HERE to Be STORED!
#             #
#             time.sleep(DELAY_SECONDS_CONTENT_LOADING)
            
#             save_processed_html()
# #             debug_breakpoint(driver.current_url, prompt_message)
            
#             # reflesh webElement after webpage reload
#             part_elms = sync_get_elements_by_xpath("//*[@id='tree-concepts']/ol/*")

#         # go back to Lesson List
#         back_link = sync_get_element_by_xpath("//h3[@class='_sidebar--header--2BaH_']/a") 
#         back_link.click()
        
#         # click open the completed lessons
#         expand_all_complete_lessons()
        
#         # reflesh webElement after webpage reload
#         lesson_elms = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")

#     # reflesh webElement after webpage reload
#     topic_elms = sync_get_elements_by_xpath("//*[@id='tree-core-curriculum']/*")
    

In [ ]:
# link_topic.click() 

In [ ]:
# expand_all_complete_lessons()



In [ ]:
# element = driver.find_element_by_xpath("//*[h3='lesson 1']")
# driver.execute_script("return arguments[0].scrollIntoView(true);", element)

In [ ]:
# driver.set_window_size(full_page_width,full_page_height)

el = driver.find_element(By.TAG_NAME, "body")
el.screenshot("main.png")


In [ ]:
# e_header = sync_get_elements_by_xpath(driver, wait, "//div[@class='css-4nu87b']")
# e_content = sync_get_elements_by_xpath(driver, wait, "//div[@class='css-dmdipe']")

# header_height = int(e_header[0].get_attribute("scrollHeight"))
# content_height = int(e_content[0].get_attribute("scrollHeight"))

# full_page_height = content_height+header_height

# full_page_width = int(e_header[0].get_attribute("scrollWidth"))